In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns
import emoji
import re
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

In [ ]:
## Loading chat data
file = open("chat.txt")

In [ ]:
data = file.read()

In [ ]:
## First have to split the data based on the date and time.
## I will use the regex to do so
pattern = "\d{1,2}/\d{1,2}/\d{1,2},\s\d{1,2}:\d{1,2}\s.M"

In [ ]:
## Finding the all messages.
message = re.split(pattern, data)[1:]
## First line is empty string

In [ ]:
## Now finding the all dates and times
dates = re.findall(pattern, data)

In [ ]:
## Checking the size of dates and messges
len(dates), len(message)

In [ ]:
## Not putting both in data frame.
df = pd.DataFrame({'date':dates, "message":message})

In [ ]:
# df.shape

In [ ]:
df.head()

In [ ]:
## Converting the date columns into date times
## Cleaning the date columns
df['date'] = pd.to_datetime(df['date'].str.replace(',',''))

In [ ]:
## Extracting the date and time individually
df['year'] = df['date'].dt.year

In [ ]:
## Extracing the month in the form of month string.
df['month'] = df['date'].dt.month_name()

In [ ]:
## Date
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

In [ ]:
## Now splitting the sender and their message respectively
df.head()

In [ ]:
## Now splitting the message with respece to sender and their messages.
print(df['message'][3])
print(df['message'][2])
print(df['message'][40])

In [ ]:
## Function to find mobile number and names
def find_sender(x):
    pattern = "^\s-\s([\w\s\d+]+):"
    res = re.search(pattern, x)
    if res == None:
        return ""
    else:
        return res[1]

In [ ]:
df['sender'] = df['message'].apply(lambda x : find_sender(x))

In [ ]:
## The message which have no sender means
## they are function message like, group creation, member added
## So removing such kind of message
indices = df[df['sender'] == ""].index
df.drop(indices, inplace = True)

In [ ]:
## Now seperating the text from message
pattern = "^\s-\s[\w\s\d+]+:"
df['text'] = df['message'].apply(lambda x : re.split(pattern, x)[1])\
            .str.replace('\\n','').str.strip().str.lower()

In [ ]:
df.drop('message', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
spammer = df[(df['text'] == 'this message was deleted') | (df['text'] == 'you deleted this message')]['sender'].value_counts()

In [ ]:
spammer.reset_index().rename({"index":"Sender","sender":"Frequency"}, axis=1)